In [1]:
import pandas as pd
import numpy as np

## Loads the datafile

In [2]:
xlsx = pd.ExcelFile('OBTestData.xlsx')
OBData = pd.read_excel(xlsx, 'Sheet1')

In [3]:
xlsx = pd.ExcelFile('entryCutoff.xlsx')
entryCutoff = pd.read_excel(xlsx, 'Sheet1')
#entryCutoff

## Transform the datecolumns to Datetime

In [4]:
OBData['ObProcessDate'] = pd.to_datetime(OBData['ObProcessDate'])
OBData['Order create dt'] = pd.to_datetime(OBData['Order create dt'])
OBData['FactoryShipDt'] = pd.to_datetime(OBData['FactoryShipDt'])

## Calculates the Business days from the order creation date up to today

In [5]:
A = [d.date() for d in OBData['Order create dt']]
B = [d.date() for d in OBData['ObProcessDate']]
OBData['Aging Days in Business Days'] = np.busday_count(A, B)

## Calculates the Calendar days from the order creation date up to today

In [6]:
OBData['Aging Days in Calendar'] =  OBData['ObProcessDate'] - OBData['Order create dt']

## Identifies the order still open at plant based on the delivery number

In [7]:
di = {True: 'Open in Plant', False: 'In Transit to Country'}
obj = pd.Series(OBData['FactoryShipDt'])
OBData['SupplyStatus'] = pd.isnull(obj)
OBData['SupplyStatus'] = pd.isnull(obj)
OBData['SupplyStatus'] = OBData['SupplyStatus'].map(di) 

## Calculates the amount of Revenue Orders availability in the plant 

In [8]:
availability = OBData[ ['SupplyStatus', 'EGNetUSDAmount']]
df1 = availability.groupby(['SupplyStatus']).sum()/1000000


## Calculates the amount of Revenue entered in each month

In [9]:
agingBusinesDays = OBData[ ['Order create dt', 'EGNetUSDAmount']]
createDate = agingBusinesDays.set_index('Order create dt').groupby(pd.Grouper(freq='M'))['EGNetUSDAmount'].sum().reset_index()
createDate['EGNetUSDAmount'] = createDate['EGNetUSDAmount']/1000000
df2 = createDate

## Calculates the amount of orders already shipped from factory

In [10]:
agingShip = OBData[ ['FactoryShipDt', 'EGNetUSDAmount']]
shipDate = agingShip.set_index('FactoryShipDt').groupby(pd.Grouper(freq='M'))['EGNetUSDAmount'].sum().reset_index()
shipDate['EGNetUSDAmount'] = shipDate['EGNetUSDAmount']/1000000
df3 = shipDate

## Calculates the amount of revenue per  Country

In [11]:
region = {'Brazil': 'Brazil', 'Argentina': 'Argentina', 'Mexico':'Mexico', 'Colombia':'Colombia', 'Chile':'Chile',
         'Peru':'Peru', 'Puerto Rico':'Puerto Rico', 'Ecuador': 'LA18', 'Guatemala':'LA18', 'Honduras':'LA18',
         'Panama': 'LA18', 'Costa Rica':'LA18', 'El Salvador': 'LA18', 'Belize': 'LA18', 'Nicaragua':'LA18',
         'Jamaica':'LA18', 'Paraguay': 'LA18', 'Uruguay':'LA18', 'Latin America HQ': 'LA18' }



OBData['SalesOrg'] = OBData['FinanceGeo'].map(region) 
laRegion = OBData[ ['SalesOrg', 'EGNetUSDAmount']]

df4 = laRegion.groupby(['SalesOrg']).sum()/1000000

## Validates the orders entered before the entry cutoff

In [12]:
cutoff = dict(zip(entryCutoff.Country, entryCutoff.EntryCutoff))

di1 = {True: 'After', False: 'Before'}


OBData['Cutoff_Date'] = OBData['SalesOrg'].map(cutoff)
OBData['Cutoff_Date'] = pd.to_datetime(OBData['Cutoff_Date'])

OBData['Cutoff_Status'] = OBData['Order create dt'] > OBData['Cutoff_Date']
OBData['Cutoff_Status'] = OBData['Cutoff_Status'].map(di1)
OpenPlant = OBData[['Order create dt', 'Cutoff_Date', 'SalesOrg', 'Cutoff_Status','EGNetUSDAmount']]
df5 = OpenPlant.groupby(['Cutoff_Status']).sum()/1000000


## Writes an excel file with the analysis

In [13]:

with pd.ExcelWriter('output.xlsx') as writer: 
     df1.to_excel(writer, sheet_name='Sheet_name_1')
     df2.to_excel(writer, sheet_name='Sheet_name_1', startrow=5)
     df3.to_excel(writer, sheet_name='Sheet_name_1', startrow=15)
     df4.to_excel(writer, sheet_name='Sheet_name_1', startrow=25)
     df5.to_excel(writer, sheet_name='Sheet_name_1', startrow=35)

## Validation

In [14]:
display(df1)
display(df2)
display(df3)
display(df4)
display(df5)

,EGNetUSDAmount
SupplyStatus,
In Transit to Country,16.586487
Open in Plant,17.582528


,Order create dt,EGNetUSDAmount
0,2018-12-31,0.009829
1,2019-01-31,0.000000
2,2019-02-28,0.000000
3,2019-03-31,0.060620
4,2019-04-30,1.737107
5,2019-05-31,7.694104
6,2019-06-30,24.603008
7,2019-07-31,0.064349


,FactoryShipDt,EGNetUSDAmount
0,2019-03-31,0.040885
1,2019-04-30,1.279111
2,2019-05-31,1.405930
3,2019-06-30,12.226310
4,2019-07-31,1.634251


,EGNetUSDAmount
SalesOrg,
Argentina,4.053330
Brazil,9.056008
Chile,4.454329
Colombia,2.029157
LA18,1.389621
Mexico,11.539559
Peru,0.541702
Puerto Rico,1.105215


,EGNetUSDAmount
Cutoff_Status,
After,2.292758
Before,31.876256
